<a href="https://colab.research.google.com/github/kizakiza2501/20201010_Digit_Recognizer/blob/main/20201024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SIGNATE 手書き数字分類
 - https://signate.jp/competitions/100/

In [1]:
import os
import glob
import pandas as pd
from pandas import DataFrame
import numpy as np

import tensorflow as tf
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator

from sklearn.model_selection import train_test_split

In [2]:
path = '/content/drive/My Drive/signate/20201023_degit_recognizer_100'
list = glob.glob(path + '/data/*')

In [3]:
list

['/content/drive/My Drive/signate/20201023_degit_recognizer_100/data/train_master.tsv',
 '/content/drive/My Drive/signate/20201023_degit_recognizer_100/data/sample_submit.tsv',
 '/content/drive/My Drive/signate/20201023_degit_recognizer_100/data/test.zip',
 '/content/drive/My Drive/signate/20201023_degit_recognizer_100/data/train.zip',
 '/content/drive/My Drive/signate/20201023_degit_recognizer_100/data/20201024.ipynb']

In [4]:
unzip '/content/drive/My Drive/signate/20201023_degit_recognizer_100/data/test.zip'

In [5]:
unzip '/content/drive/My Drive/signate/20201023_degit_recognizer_100/data/train.zip'

In [6]:
df_train_master = pd.read_csv(path + '/data/train_master.tsv', sep='\t')
df_sample_submition = pd.read_csv(path + '/data/sample_submit.tsv', sep='\t', header=None)

In [7]:
df_train_master.head()

,file_name,category_id
0,train_0.jpg,5
1,train_1.jpg,0
2,train_2.jpg,4
3,train_3.jpg,1
4,train_4.jpg,9


In [8]:
df_sample_submition.head()

,0,1
0,test_0.jpg,2
1,test_1.jpg,4
2,test_2.jpg,5
3,test_3.jpg,4
4,test_4.jpg,3


In [9]:
train = []
test = []

# 学習データ
for row in df_train_master.itertuples():
  temp_img = load_img('train/' + row[1], color_mode = "grayscale")
  train.append(img_to_array(temp_img))

train = np.asarray(train)

In [10]:
train[0].shape

(28, 28, 1)

In [12]:
# One-Hot-Encodingで変換する
y = df_train_master['category_id']
y = tf.keras.utils.to_categorical(y, num_classes=10)

In [13]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [14]:
# 学習データと、バリデーションデータを分割
x_train, x_val, y_train, y_val = train_test_split(train, y, test_size=0.2)

In [15]:
# テストデータ
for row in df_sample_submition.itertuples():
  temp_img = load_img('test/' + row[1], color_mode = "grayscale")
  test.append(img_to_array(temp_img))

test = np.asarray(test)

In [16]:
# 学習データ向けのジェネレータ
train_data_gen = ImageDataGenerator(
    rescale = 1.0/255.0,
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=20,
    zoom_range = 0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=False,
    vertical_flip=False
)

train_generator = train_data_gen.flow(
    # generator向けに4次元へ変換する
    # grascaleなので、最後はチャネル数は1
    x_train.reshape(x_train.shape[0], 28, 28, 1),
    y_train,
    batch_size=32,
)

In [17]:
# テストデータ向けのジェネレータ
# テストデータは拡張しない（正規化のみ行う）
test_data_gen = ImageDataGenerator(
    rescale=1.0/255.0
)

test_generator = test_data_gen.flow(
    # generator向けに4次元へ変換する
    # shuffleがデフォルトTrueなので、明示的にFalseを設定する
    # （shuffleすると、sumission.csvの順番とずれてしまう）
    test.reshape(test.shape[0], 28, 28, 1),
    batch_size=1,
    shuffle=False
)

In [18]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if logs.get('accuracy') > 0.99:
      print('Reached 99% accracy so cancelling training.')
      self.model.stop_traning = True

callbacks = myCallback()

In [19]:
'''
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, 7, activation='relu', padding='same', input_shape=[28, 28, 1]),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])
'''
model = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(32, (5,5), activation='relu', kernel_initializer='he_uniform',input_shape=(28, 28, 1)),
            tf.keras.layers.MaxPooling2D(2, 2),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Conv2D(64, (5,5), activation='relu'),
            tf.keras.layers.MaxPooling2D(2, 2),
            tf.keras.layers.Dropout(0.25),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(10, activation='softmax')
    ])
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = tf.keras.optimizers.SGD(lr=0.1, momentum=0.9),
    metrics=['accuracy']
)

In [20]:
tf.keras.backend.clear_session()

history = model.fit_generator(
    train_generator,
    steps_per_epoch=x_train.shape[0] // 32, 
    epochs = 15,
    verbose = 1,
    validation_data = (x_val, y_val),
    callbacks=[callbacks]
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
1500/1500 [==============================] - 18s 12ms/step - loss: 0.4224 - accuracy: 0.8667 - val_loss: 429.2678 - val_accuracy: 0.7595
Epoch 2/15
1500/1500 [==============================] - 17s 12ms/step - loss: 0.1832 - accuracy: 0.9439 - val_loss: 665.8669 - val_accuracy: 0.6147
Epoch 3/15
1500/1500 [==============================] - 17s 12ms/step - loss: 0.1441 - accuracy: 0.9558 - val_loss: 407.0125 - val_accuracy: 0.7509
Epoch 4/15
1500/1500 [==============================] - 17s 11ms/step - loss: 0.1245 - accuracy: 0.9624 - val_loss: 446.0224 - val_accuracy: 0.6845
Epoch 5/15
1500/1500 [==============================] - 17s 11ms/step - loss: 0.1126 - accuracy: 0.9656 - val_loss: 168.6465 - val_accuracy: 0.8246
Epoch 6/15
1500/1500 [==============================] - 17s 12ms/step - loss: 0.1087 - accuracy: 0.9673 - val_loss: 117.2469 - val_accuracy: 0.8378
Epoch 7/15
1500/1500 [==============

In [21]:
# np.argmaxで、10個のうち一番一番確率の大きいものを選ぶ
pred_temp = DataFrame(np.argmax(model.predict_generator(test_generator, steps = test.shape[0]), axis=1))
# pred_temp = DataFrame(np.argmax(model.predict(test), axis=1))

Instructions for updating:
Please use Model.predict, which supports generators.


In [22]:
df_sample_submition[1] = pred_temp

In [23]:
df_sample_submition.head()

,0,1
0,test_0.jpg,7
1,test_1.jpg,2
2,test_2.jpg,1
3,test_3.jpg,0
4,test_4.jpg,4


In [24]:
df_sample_submition.to_csv('result.tsv', sep='\t', index=False)